In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
import os
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import dask_image.imread
import numpy as np
from skimage import feature, morphology, img_as_float
import scipy.ndimage as ndi
from random import random
import sklearn
from sklearn.decomposition import PCA
import cc3d
import pandas as pd
import seaborn as sns
import math

import cv2

from tqdm import tqdm_notebook as tqdm

In [ ]:
def get_all_mito_comp(mito_np, la_np, erosion_iteration = 4, opening=False, thresh=100):
    """
    Calculate the mitochondrial compartments

    Parameters
    ----------
    mito_np : ndarray
        Labels of mitochondria
    la_np : ndarray
        Labels of lamellar cristae
    erosion_iteration : int
        Iteration times of erosion operation applied to mitochondria labels
    opening: bool, default False
        When True, opening operation is applied to compartments before cc3d.connected_components is applied
    thresh: int, default 100
        Threshold of compartments volume

    Returns
    -------
    final array: ndarray
        Compartment labels starting with 1 for each mitochondrion
    """
    
    connectivity = 6 
    diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
    labels = cc3d.connected_components(mito_np, connectivity = connectivity)
    mito_q, mito_num = np.unique(labels, return_counts=True)
    la_np = 1*(la_np>0)
    final_array = np.zeros_like(la_np)
    for i in tqdm(range(1, len(mito_q))):
        if mito_num[i] <1000:
            continue
        else:
            mito1 = np.where(labels==i, labels, 0)
            mito1_eroded =ndi.binary_erosion(1*(mito1>0), diamond, iterations=erosion_iteration)
            mito1_separated = ((mito1_eroded*1 - la_np)>0)*1
            if opening:
                mito1_separated =ndi.binary_erosion(mito1_separated, diamond, iterations=1)
                mito1_separated =ndi.binary_dilation(mito1_separated, diamond, iterations=1)
            labels_comp = cc3d.connected_components(mito1_separated, connectivity = connectivity)
            lacomp_q, la_comp_num = np.unique(labels_comp,return_counts=True)
            la_comp_num_sorted_idx = np.argsort(la_comp_num)[::-1]
            la_comp_temp = np.zeros_like(labels_comp)
            for j, idx in enumerate(la_comp_num_sorted_idx):
                if j==0:
                    continue
                if la_comp_num[idx] < thresh:
                    break
                else:
                    la_comp_temp = np.where(labels_comp == idx, j, la_comp_temp)
            final_array = final_array + la_comp_temp
    return final_array

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)


mito= mito[:, :692, :2275]

mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255

    
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)



la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/compartment_lamellar_002/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )

In [ ]:
filename_pattern = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito/', '*png')
mito = dask_image.imread.imread(filename_pattern)
mito_np = np.zeros(mito.shape, dtype=int)
for i in range(len(mito)):
    mito_np[i] = (mito[i].compute()>0)*255
filename_pattern_l = os.path.join('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/lamellar_with_postprocessing//', '*png')
la = dask_image.imread.imread(filename_pattern_l)
la_np = np.zeros(la.shape, np.uint8)
for i in range(len(la)):
    la_np[i] = (la[i].compute()>0)*255

In [ ]:
f_array = get_all_mito_comp(mito_np, la_np)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/compartment_lamellar/"

os.makedirs(out_dir, exist_ok=True)

for i in range(f_array.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        f_array[i]
    )